This notebooks is for aligning netcdf grids

In [1]:
import os
import numpy
import xarray as xr
from scipy.misc import imresize

import warnings
warnings.filterwarnings("ignore")

In [15]:
ds_coarse = xr.open_dataset('../example_data/GLDAS/FORCING/2019022600.LDASIN_DOMAIN1')
ds_fine = xr.open_dataset('../example_data/Hydro_Estimator_netcdf/Hydro_Estimator201908312.nc')

In [16]:
ds_coarse

<xarray.Dataset>
Dimensions:     (Time: 1, south_north: 56, west_east: 72)
Dimensions without coordinates: Time, south_north, west_east
Data variables:
    lat         (south_north, west_east) float32 ...
    lon         (south_north, west_east) float32 ...
    Times       |S20 ...
    valid_time  (Time) datetime64[ns] ...
    T2D         (Time, south_north, west_east) float64 ...
    Q2D         (Time, south_north, west_east) float64 ...
    U2D         (Time, south_north, west_east) float64 ...
    V2D         (Time, south_north, west_east) float64 ...
    PSFC        (Time, south_north, west_east) float64 ...
    RAINRATE    (Time, south_north, west_east) float64 ...
    SWDOWN      (Time, south_north, west_east) float64 ...
    LWDOWN      (Time, south_north, west_east) float64 ...

Look at the shapes of these datasets

In [18]:
shape_fine = ds_fine.lat.values.shape
shape_coarse = ds_coarse.RAINRATE.shape

print('Fine resolution shape: %s' % str(shape_fine))
print('Coarse resolution shape: %s' % str(shape_coarse))

Fine resolution shape: (2, 3111, 8001)
Coarse resolution shape: (1, 56, 72)


Take a look at the data and make sure there are non-nan values

In [61]:
ds_coarse.Rainf_f_tavg

<xarray.DataArray 'Rainf_f_tavg' (time: 1, lat: 600, lon: 1440)>
[864000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 -59.875 -59.625 -59.375 ... 89.375 89.625 89.875
  * lon      (lon) float32 -179.875 -179.625 -179.375 ... 179.625 179.875
  * time     (time) datetime64[ns] 2019-02-26
Attributes:
    units:          kg m-2 s-1
    standard_name:  precipitation_flux
    long_name:      Total precipitation rate
    cell_methods:   time: mean
    vmin:           0.0
    vmax:           0.009105

In [62]:
arr = ds_coarse.Rainf_f_tavg.values[0]

In [63]:
numpy.nanmin(ds_coarse.Rainf_f_tavg.values)

0.0

In [64]:
numpy.nanmax(ds_coarse.Rainf_f_tavg.values)

0.0091049997

Get the corner lat/lon for the target array

In [65]:
xmin = ds_fine.lon.values.min()
xmax = ds_fine.lon.values.max()
ymin = ds_fine.lat.values.min()
ymax = ds_fine.lat.values.max()

Select all values in the source array within this range

In [66]:
lons = ds_coarse.lon.values
lats = ds_coarse.lat.values

In [67]:
# get the coordinates in the source array that match the range of the target
inner_lons = numpy.where((lons < xmax) & (lons > xmin))
inner_lats = numpy.where((lats < ymax) & (lats > ymin))

In [68]:
inner_lons

(array([87, 88, 89]),)

In [69]:
inner_lats

(array([325, 326]),)

In [70]:
# define index bounds for selecting data using numpy indexing
llon = inner_lons[0].min()
ulon = inner_lons[0].max() + 1
llat = inner_lats[0].min()
ulat = inner_lats[0].max() + 1

In [71]:
# select the data from the target for this range
arr = ds_coarse['Rainf_f_tavg'][0].values
subset = arr[llon:ulon, llat:ulat]
print(subset)

[[ nan  nan]
 [ nan  nan]
 [ nan  nan]]


In [141]:
# replace nan with 0
subset = numpy.nan_to_num(subset)

# get the max value used for converting the interpolated data back into
# the original range
subset_max = subset.max()

In [142]:
subset

array([[    0.,  1000.],
       [    0.,     0.],
       [  200.,     0.]], dtype=float32)

In [143]:
# insert some random values (testing only)
subset[0][1] = 1000
subset[2][0] = 200

In [144]:
subset

array([[    0.,  1000.],
       [    0.,     0.],
       [  200.,     0.]], dtype=float32)

In [177]:
# resize the array to the shape of the target
d = imresize(subset,
         (10,10),
         interp='bilinear',
         mode=None)


In [178]:
# convert from uint8 back into our original range of data
d = d/255 * subset_max
d

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

Create a function to do these steps

In [185]:
def regrid_variables(source, target, variables):
    ds_fine = xr.open_dataset(source)
    ds_coarse = xr.open_dataset(target)
    
    xmin = ds_fine.lon.values.min()
    xmax = ds_fine.lon.values.max()
    ymin = ds_fine.lat.values.min()
    ymax = ds_fine.lat.values.max()
    
    lons = ds_coarse.lon.values
    lats = ds_coarse.lat.values

    # get the coordinates in the source array that match the
    # range of the target
    inner_lons = numpy.where((lons < xmax) & (lons > xmin))
    inner_lats = numpy.where((lats < ymax) & (lats > ymin))
    
    # define index bounds for selecting data using numpy indexing
    llon = inner_lons[0].min()
    ulon = inner_lons[0].max() + 1
    llat = inner_lats[0].min()
    ulat = inner_lats[0].max() + 1
    
    regridded_data = {}
    for v in variables:
    
        # select the data from the target for this range
        arr = ds_coarse[v][0].values
        subset = arr[llon:ulon, llat:ulat]

        # replace nan with 0
        subset = numpy.nan_to_num(subset)

        # get the max value used for converting the interpolated data back into
        # the original range
        subset_max = subset.max()


        # resize the array to the shape of the target
        d = imresize(subset,
                 (10,10),
                 interp='bilinear',
                 mode=None)


        # convert from uint8 back into our original range of data
        d = d/255 * subset_max
        regridded_data[v] = d
    return regridded_data

In [187]:
target = '../example_data/GLDAS/FORCING/2019022600.LDASIN_DOMAIN1'
# source = '../example_data/GLDAS/input_files/GLDAS_NOAH025_3H.A20190226.0300.021.nc4'
variables = ['T2D',
             'Q2D',
             'U2D',
             'V2D',
             'RAINRATE',
             'SWDOWN',
             'LWDOWN',
             'PSFC']
inputdir = '../example_data/GLDAS/input_files'
source_files = os.listdir(inputdir)
for f in source_files:
    print('\n%s' % f)
    source = os.path.join(inputdir, f)
    dat = regrid_variables(source, target, variables)
    for k,v in dat.items():
        max_val = dat[k].max()
        if max_val > 0:
            print(dat[k])
        else:
            print('%s: maximum value is 0' % (k) )


GLDAS_NOAH025_3H.A20190226.0000.021.nc4
T2D: maximum value is 0
Q2D: maximum value is 0
U2D: maximum value is 0
V2D: maximum value is 0
RAINRATE: maximum value is 0
SWDOWN: maximum value is 0
LWDOWN: maximum value is 0
PSFC: maximum value is 0

GLDAS_NOAH025_3H.A20190226.0300.021.nc4
T2D: maximum value is 0
Q2D: maximum value is 0
U2D: maximum value is 0
V2D: maximum value is 0
RAINRATE: maximum value is 0
SWDOWN: maximum value is 0
LWDOWN: maximum value is 0
PSFC: maximum value is 0

GLDAS_NOAH025_3H.A20190226.0600.021.nc4
T2D: maximum value is 0
Q2D: maximum value is 0
U2D: maximum value is 0
V2D: maximum value is 0
RAINRATE: maximum value is 0
SWDOWN: maximum value is 0
LWDOWN: maximum value is 0
PSFC: maximum value is 0

GLDAS_NOAH025_3H.A20190226.0900.021.nc4
T2D: maximum value is 0
Q2D: maximum value is 0
U2D: maximum value is 0
V2D: maximum value is 0
RAINRATE: maximum value is 0
SWDOWN: maximum value is 0
LWDOWN: maximum value is 0
PSFC: maximum value is 0

GLDAS_NOAH025_3H.A2